<a href="https://colab.research.google.com/github/harperd/machine-learning/blob/master/notebooks/neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Network

Create a neural network to recognize hand-written digits (0 to 9).

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
import google.colab as colab
import scipy.optimize as opt
import io

from scipy.io import loadmat

# Allow saving our graphs in the notebook
%matplotlib inline

style.use('dark_background')

In [2]:
mat_file = colab.files.upload()
!ls -l

Saving ex4data1.mat to ex4data1.mat
total 7340
-rw-r--r-- 1 root root 7511764 Sep 14 16:26 ex4data1.mat
drwxr-xr-x 1 root root    4096 Aug 27 16:17 sample_data


In [3]:
mat_data = loadmat('ex4data1.mat')
mat_data

{'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [4]:
X = mat_data['X']
y = mat_data['y']

print(f'X Shape: {X.shape}')
print(f'y Shape: {y.shape}')

X Shape: (5000, 400)
y Shape: (5000, 1)


![Hand written numbers](https://github.com/harperd/machine-learning/blob/master/images/ex3-1.png?raw=1)

In [6]:
def xavier(prev_layer_size):
    return np.sqrt(1 / prev_layer_size)

def initialize_parameters(layers_dims):
    np.random.seed(1)               
    parameters = {}
    
    # Get the number of layers
    L = len(layers_dims)            
    
    # For each layer initalize the weights and bias vector
    for l in range(1, L): 
        parameters["W" + str(l)] = np.random.randn(
            layers_dims[l], layers_dims[l - 1]) * xavier(layers_dims[l - 1])
        parameters["b" + str(l)] = np.zeros((layers_dims[l], 1))

        assert parameters["W" + str(l)].shape == (
            layers_dims[l], layers_dims[l - 1])
        assert parameters["b" + str(l)].shape == (layers_dims[l], 1)

    return parameters
  
layer_dims = [400, 25, 10]

initialize_parameters(layer_dims)

{'W1': array([[ 0.08121727, -0.03058782, -0.02640859, ...,  0.04603076,
         -0.0017684 ,  0.10553025],
        [-0.0653267 ,  0.00381902,  0.01836159, ..., -0.03514602,
          0.03627753, -0.01621021],
        [ 0.04071716,  0.0390235 , -0.07320268, ..., -0.07244958,
          0.03897459, -0.05431505],
        ...,
        [-0.05908141, -0.00380526, -0.041776  , ..., -0.03318272,
         -0.07156263,  0.07103446],
        [ 0.00655865, -0.03714021,  0.03185974, ...,  0.02187983,
          0.00843188, -0.02517834],
        [-0.04064453,  0.00992197,  0.00546658, ..., -0.05070719,
         -0.00313481, -0.07189349]]),
 'W2': array([[-0.02449478,  0.04563396, -0.07046103, -0.16611069, -0.05221796,
          0.03387085,  0.13472462, -0.06544032, -0.06105983,  0.10497307,
         -0.11683738, -0.04555045, -0.10658141, -0.16007361,  0.2059303 ,
         -0.13194443, -0.00922755,  0.04230281, -0.1256229 ,  0.12226152,
          0.09363321,  0.15892917,  0.09406254,  0.17882511,  0.3